# Protein structures in RCSB PDB

## Introduction
RCSB PDB (Research Collaboratory for Structural Bioinformatics Protein Data Bank) is a database widely used by scientists in many fields such as biochemistry. It stores structures of biomolecules, mostly proteins, that have been obtained by experimental methods.


### Hypotheses

1) number of added structures rises exponentaly
2) electron microscopy is getting more popular
3) there are mostly protein structures in the database
4) most of the protein structures are enzymes, mostly hydrolases
5) most of the molecules are produced in bacteria but come from different sources

In [ ]:
# importing modules
import pandas as pd # dataframes
import altair as alt # plotting, Vega-Altair version 5.5.0 was used
import biotite.database.rcsb as rcsb # api for rcsb database

## API - RCSB PDB

### 1) Number of all structures
number of added structures rises exponentaly

In [ ]:
# number of all structures added before 1990
# obtaining data from rcsb pdb through api

query_old=rcsb.FieldQuery("rcsb_accession_info.initial_release_date", range_closed=('1960-01-01', '1989-12-31'))
all_old=rcsb.count(query_old)
all_old

In [ ]:
# number of all added structures from 1990 per year

years=list(range(1990,2026))
counted_years_all=[]
counted_structures_all=[]

# obtaining data from rcsb pdb through api

for year in years:
    query_year_all=rcsb.FieldQuery("rcsb_accession_info.initial_release_date", range_closed=(f'{year}-01-01', f'{year}-12-31'))
    counted_structures_all.append(rcsb.count(query_year_all))
    counted_years_all.append(year)

# saving data to dataframe

data_rcsb_all= pd.DataFrame(
    {'publicationYear': counted_years_all,
     'countedStructures': counted_structures_all
    })

In [ ]:
# plotting a bar chart of number of added structures per year

yearly_all=alt.Chart(data_rcsb_all
).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3,
    color='lightseagreen'   
).encode(
    x=alt.X('publicationYear:O'),
    y=alt.Y('countedStructures:Q')
)

In [ ]:
# plotting a line chart of cumulative number of all added structures

cumulative=alt.Chart(data_rcsb_all, title='Cumulative number of published structures in the RCSB PDB'
).transform_window(
    sort=[{"field": "publicationYear"}],
    summed='sum(countedStructures)'
).transform_calculate(
    cumulative_count=alt.datum.summed + all_old
).mark_line(
    color='crimson'
).encode(
    x=alt.X("publicationYear:O").title('Publication year'),
    y=alt.Y("cumulative_count:Q").stack(False).title('Cumulative number of structures')
)

In [ ]:
# showing the two charts combined

(cumulative + yearly_all).configure(background='oldlace')

### 2) Experimental metods
electron microscopy is getting more popular

In [ ]:
# defining function to obtain data from rcsb pdb through api

def get_data_yearly (years:list, attribute:str, values:list, column_name:str):
    """
    fetches data from rcsb pdb and saves it into a dataframe
    searching by e.g, different experimental methods, structure types
    searches for number of structures per year
    possible attributes and values can be found at https://search.rcsb.org/structure-search-attributes.html

    Args:
        years (list): years to search for
        attribute (str): atttibutes to search for
        values (list): values to search for
        column_name (str): name of column in the dataframe for attribute

    Returns:
        (pd.DataFrame): numbers of structures per year for selected attribute and values
    """
    
    counted_structures=[]
    counted_years=[]
    counted_values=[]
    for year in years:
        query_year=rcsb.FieldQuery("rcsb_accession_info.initial_release_date", range_closed=(f'{year}-01-01', f'{year}-12-31'))
        for value in values:
            query_attribute = rcsb.FieldQuery(attribute, exact_match=value)
            counted_structures.append(rcsb.count(query_attribute & query_year))
            counted_years.append(year)
            counted_values.append(value)
    return pd.DataFrame(
        {column_name: counted_values,
         'publicationYear': counted_years,
         'countedStructures': counted_structures
        })

In [ ]:
# fetching data from rcsb pdb
# number of added structures obtained by selected methods from 1990 per year
# runs for about a minute

methods=["X-RAY DIFFRACTION", "SOLUTION NMR", "ELECTRON MICROSCOPY"]
data_rcsb_method=get_data_yearly(years, "exptl.method",  methods, 'experimentalTechnique')

In [ ]:
# plotting a bar chart of number of added structures obtained by selected methods per year

yearly_method=alt.Chart(data_rcsb_method, title='Number of published structures per year by experimental method'
).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3   
).encode(
    x=alt.X('publicationYear:O').title('Publication year'),
    y=alt.Y('countedStructures:Q').title('Number of structures'),
    color=alt.Color('experimentalTechnique:N').title('Method').scale(scheme='set2')
)
yearly_method.configure(background='oldlace')

### 3) Types of structures
there are mostly protein structures in the database

In [ ]:
# fetching data from rcsb pdb
# number of added structures by structure type from 1990 per year
# runs for about a minute

types=["Protein", "DNA", "RNA", "NA-hybrid", "Other"]
data_rcsb_type=get_data_yearly (years, "entity_poly.rcsb_entity_polymer_type", types, 'macromoleculeType')

In [ ]:
# plotting a bar chart of number of added structures by structure type per year

yearly_type=alt.Chart(data_rcsb_type, title='Number of published structures per year by structure type'
).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3   
).encode(
    x=alt.X('publicationYear:O').title('Publication year'),
    y=alt.Y('countedStructures:Q').title('Number of structures'),
    color=alt.Color('macromoleculeType:N', sort=alt.EncodingSortField(field="macromoleculeType", op='count', order='descending')).title('Structure type').scale(scheme='set2'),
    order=alt.Order('countedStructures', sort='descending')
)
yearly_type.configure(background='oldlace')

### 4) Classification of protein structures
most of the protein structures are enzymes, mostly hydrolases

In [ ]:
# defining function to obtain data from rcsb pdb through api

def get_data_all (attribute:str, values:list, column_name:str):
    """
    fetches data from rcsb pdb and saves it into a dataframe
    searching by e.g, different experimental methods, structure types
    searches for number of structures in total
    possible attributes and values can be found at https://search.rcsb.org/structure-search-attributes.html

    Args:
        attribute (str): atttibutes to search for
        values (list): values to search for
        column_name (str): name of column in the dataframe for attribute

    Returns:
        (pd.DataFrame): numbers of structures for selected attribute and values
    """
    
    counted_structures=[]
    counted_values=[]
    for value in values:
        query_attribute = rcsb.FieldQuery(attribute, exact_match=value)
        counted_structures.append(rcsb.count(query_attribute))
        counted_values.append(value)
    return pd.DataFrame(
        {column_name: counted_values,
         'countedStructures': counted_structures
        })

In [ ]:
# number of all protein structures
# obtaining data from rcsb pdb through api

query_protein=rcsb.FieldQuery("entity_poly.rcsb_entity_polymer_type", exact_match='Protein')
all_protein=rcsb.count(query_protein)
all_protein

In [ ]:
# fetching data from rcsb pdb
# number of added enzyme structures by enzyme class (search by enzyme class number)

enzymes=['1','2','3','4','5','6','7']
data_rcsb_enzyme=get_data_all ("rcsb_polymer_entity.rcsb_ec_lineage.id", enzymes, 'enzymeClass')

In [ ]:
# adding a collumn with enzyme class name into the dataframe

data_rcsb_enzyme.insert(0, "className", ['Oxidoreductase', 'Transferase', 'Hydrolase', 'Lyase', 'Isomerase', 'Ligase', 'Translocase'])

In [ ]:
# calculating the number of all other proteins that do not belong to any of the enzyme class and adding the result into the dataframe

others = all_protein-sum(data_rcsb_enzyme['countedStructures'])
data_rcsb_enzyme.loc[7] = ['Other proteins', None, others]

In [ ]:
# plotting a pie chart of number of enzymes by enzyme class and number of other proteins

all_enzyme=alt.Chart(data_rcsb_enzyme, title='Number of published structures by enzyme class'
).mark_arc().encode(
    theta=alt.Theta("countedStructures").stack(True),
    color=alt.Color("className:N", sort=None ,title=('Enzyme class')).scale(scheme='set2'),
    order=alt.Order('countedStructures')
)

chart_enzyme=all_enzyme.mark_arc(outerRadius=120)
text_enzyme=all_enzyme.mark_text(radius=140, size=10, angle=315).encode(text="countedStructures:N")

(chart_enzyme+text_enzyme).configure(background='oldlace')

### 5) Organisms
most of the molecules are produced in bacteria but come from different sources

In [ ]:
# fetching data from rcsb pdb
# number of added structures by selected source and host organism

organisms=['Escherichia coli', 'Saccharomyces cerevisiae', 'Archaea', 'Bacteria', 'Eukaryota', 'Fungi', 'Embryophyta', 'Mammalia', 'Homo sapiens']
data_rcsb_source=get_data_all ("rcsb_entity_source_organism.taxonomy_lineage.name", organisms, 'organism')
data_rcsb_host=get_data_all ("rcsb_entity_host_organism.taxonomy_lineage.name", organisms, 'organism')

In [ ]:
# merging the data into one dataframe, Vega-Altair works well with long data format

data_rcsb_source['type']='source'
data_rcsb_host['type']='host'
data_rcsb_organism=pd.concat([data_rcsb_source, data_rcsb_host])

In [ ]:
# plotting a bar chart of number of structures by selected source and host organism

all_organism=alt.Chart(data_rcsb_organism, title=alt.TitleParams('Number of published structures by source and host organisms', anchor='middle'),
).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3   
).encode(
    x=alt.X('type:N', title='', axis=alt.Axis(labels=False, ticks=False), sort='descending'),
    y=alt.Y('countedStructures:Q').title('Number of structures'),
    color=alt.Color('type:N', title='', sort='descending').scale(scheme='set2'),
    column=alt.Column('organism:N', title='', header=alt.Header(labelOrient='bottom', labelAlign='center',labelLimit=60), sort=['organism'])
)

all_organism.configure(background='oldlace')

## Dataset from kaggle
obtained from https://www.kaggle.com/datasets/shahir/protein-data-set

In [ ]:
# loading data

data = pd.read_csv("pdb_data_no_dups.csv")

In [ ]:
# allows Vega-Altair work with big datasets

alt.data_transformers.disable_max_rows()

In [ ]:
# fixing typo in one data point, droping data with unknown publication year

data_year_fix=data.replace({'publicationYear':201}, 2014).dropna(subset=['publicationYear'])

# plotting a bar chart of number of added structures by experimental method from 1990 per year

alt.Chart(data_year_fix, title='Number of published structures per year by experimental method').mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    x=alt.X('publicationYear:O', title='Publication year'),
    y=alt.Y('count():Q', title='Number of structures'),
    color=alt.Color('experimentalTechnique:N', sort=alt.EncodingSortField(field="experimentalTechnique", op='count', order='descending')
                    ,title='Experimental method').scale(scheme='set2')
).transform_filter(
    alt.datum.publicationYear > 1990
).configure(background='oldlace')

In [ ]:
# plotting a pie chart of number of added structures by structure type

kaggle_type=alt.Chart(data).mark_arc().encode(
    theta=alt.Theta("count()"),
    color=alt.Color("macromoleculeType:N", sort=alt.EncodingSortField(field="macromoleculeType", op='count', order='descending')
                    ,title=('Macromolecule type')).scale(scheme='set2'),
    order=alt.Order('count()')
)

In [ ]:
# plotting a pie chart of number of added structures by structure classification

kaggle_class=alt.Chart(data).mark_arc().encode(
    theta=alt.Theta("count()"),
    color=alt.Color("classification:N", sort=alt.EncodingSortField(field="macromoleculeType", op='count', order='descending')
                                                                   ,title=('Classification')).scale(scheme='set2'),
    order=alt.Order('count()')
)

In [ ]:
# showing the two charts next to each other

alt.hconcat(kaggle_type, kaggle_class).resolve_scale(color='independent').properties(title='Number of published structures by structure type and classification').configure(background='oldlace')